In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
# 코드에서 nms제외해야함

In [2]:
# !python3 -m pip install --pre-upgrade tensorrt
# !pip install pycuda

In [3]:
# !python3 -m pip install --pre --upgrade tensorrt
# !apt-get install tensorrt -y
# !wget https://developer.nvidia.com/downloads/compute/machine-learning/tensorrt/10.0.0/tars/TensorRT-10.0.0.6.Linux.x86_64-gnu.cuda-11.8.tar.gz

# python3 -m pip install tensorrt-10.0.0b6-cp38-none-linux_x86_64.whl
# export PATH=/path/to/tensorrt/bin:$PATH
# export LD_LIBRARY_PATH=<TensorRT-${version}/lib>:$LD_LIBRARY_PATH
# source ~/.bashrc
# https://docs.nvidia.com/deeplearning/tensorrt/install-guide/index.html#installing-tar
# https://da2so.tistory.com/61

In [4]:
# base_name = "./deploy_pillar_sfaw_3d/dsvt_3d_bb.onnx"
# --memPoolSize=workspace:4096
# --device=1
# --buildOnly
# --tacticSources=+CUDNN,+CUBLAS,-CUBLAS_LT,+EDGE_MASK_CONVOLUTIONS \
# --minShapes=src:3000x192,set_voxel_inds_tensor_shift_0:2x170x36,set_voxel_inds_tensor_shift_1:2x100x36,set_voxel_masks_tensor_shift_0:2x170x36,set_voxel_masks_tensor_shift_1:2x100x36,pos_embed_tensor:4x2x3000x192 \
# --optShapes=src:20000x192,set_voxel_inds_tensor_shift_0:2x1000x36,set_voxel_inds_tensor_shift_1:2x700x36,set_voxel_masks_tensor_shift_0:2x1000x36,set_voxel_masks_tensor_shift_1:2x700x36,pos_embed_tensor:4x2x20000x192 \
# --maxShapes=src:35000x192,set_voxel_inds_tensor_shift_0:2x1500x36,set_voxel_inds_tensor_shift_1:2x1200x36,set_voxel_masks_tensor_shift_0:2x1500x36,set_voxel_masks_tensor_shift_1:2x1200x36,pos_embed_tensor:4x2x35000x192 


# !trtexec --onnx=./deploy_pillar_sfaw_3d_origin/deploy_pillar_sfaw_3d_origin.onnx  --saveEngine=./deploy_pillar_sfaw_3d/dsvt_3d_bb_origin.engine --verbose   --fp16

# --optShapes=src:962x192,set_voxel_inds_tensor_shift_0:2x40x36,set_voxel_inds_tensor_shift_1:2x34x36,set_voxel_masks_tensor_shift_0:2x40x36,set_voxel_masks_tensor_shift_1:2x34x36,pos_embed_tensor:4x2x962x192 \
# --optShapes=src:20000x192,set_voxel_inds_tensor_shift_0:2x1000x36,set_voxel_inds_tensor_shift_1:2x700x36,set_voxel_masks_tensor_shift_0:2x1000x36,set_voxel_masks_tensor_shift_1:2x700x36,pos_embed_tensor:4x2x20000x192 \
# --maxShapes=src:35000x192,set_voxel_inds_tensor_shift_0:2x1500x36,set_voxel_inds_tensor_shift_1:2x1200x36,set_voxel_masks_tensor_shift_0:2x1500x36,set_voxel_masks_tensor_shift_1:2x1200x36,pos_embed_tensor:4x2x35000x192 


In [5]:
import numpy as np
import tensorrt as trt
import torch
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.models import build_network
from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils
# import pycuda.driver as cuda
# import pycuda.autoinit
# import torch_tensorrt

# import pycuda.driver as cuda
# import pycuda.autoinit

fatal: detected dubious ownership in repository at '/mnt/nas2/users/eslim/workspace/OpenPCDet'
To add an exception for this directory, call:

	git config --global --add safe.directory /mnt/nas2/users/eslim/workspace/OpenPCDet


In [6]:
####### load model #######
# cfg_file = "./cfgs/dsvt_models/dsvt_plain_1f_onestage.yaml"
# cfg_from_yaml_file(cfg_file, cfg)
# if os.path.exists('./deploy_files')==False:
#     os.mkdir('./deploy_files')
# log_file = './deploy_files/log_trt.log'

cfg_file = "./pillar/config.yaml"
cfg_from_yaml_file(cfg_file, cfg)
if os.path.exists('./deploy_pillar_sfaw_2d_rev')==False:
    os.mkdir('./deploy_pillar_sfaw_2d_rev')
log_file = './deploy_pillar_sfaw_2d_rev/log_trt.log'

logger = common_utils.create_logger(log_file, rank=0)
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=1,
    dist=False, workers=8, logger=logger, training=False
)

model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)
ckpt = "./pillar/model.pth"

model.load_params_from_file(filename=ckpt, logger=logger, to_cpu=False, pre_trained_path=None)
model.eval()
model.cuda()
####### load model #######

####### read input #######
batch_dict = torch.load("/mnt/nas2/users/eslim/onnx/sfaw/input_dict.pth", map_location="cuda")
inputs = batch_dict


2024-04-19 01:52:35,061   INFO  Loading Custom dataset.
2024-04-19 01:52:35,063   INFO  Total samples for CUSTOM dataset: 0
2024-04-19 01:52:37,319   INFO  ==> Loading parameters from checkpoint ./pillar/model.pth to GPU
2024-04-19 01:52:37,470   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+255db8f
2024-04-19 01:52:37,514   INFO  ==> Done (loaded 391/391)


In [7]:
model

CenterPoint(
  (vfe): DynamicVoxelVFE(
    (pfn_layers): ModuleList(
      (0): PFNLayerV2(
        (linear): Linear(in_features=10, out_features=96, bias=False)
        (norm): BatchNorm1d(96, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (1): PFNLayerV2(
        (linear): Linear(in_features=192, out_features=192, bias=False)
        (norm): BatchNorm1d(192, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
    )
  )
  (backbone_3d): DSVT(
    (input_layer): DSVTInputLayer(
      (posembed_layers): ModuleList(
        (0): ModuleList(
          (0-3): 4 x ModuleList(
            (0-1): 2 x PositionEmbeddingLearned(
              (position_embedding_head): Sequential(
                (0): Linear(in_features=2, out_features=192, bias=True)
                (1): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
       

In [8]:
inputs_vfe = model.vfe(inputs)
print(f"Memory Allocated: {torch.cuda.memory_allocated(0) / (1024 ** 3):.2f} GB")

inputs_3d = model.backbone_3d(inputs_vfe)
print(f"Memory Allocated: {torch.cuda.memory_allocated(0) / (1024 ** 3):.2f} GB")

inputs_bev = model.map_to_bev_module(inputs_3d)
import time
start_time = time.time()
print(f"Memory Allocated: {torch.cuda.memory_allocated(0) / (1024 ** 3):.2f} GB")
inputs_2d = model.backbone_2d(inputs_bev)
print(f"Memory Allocated: {torch.cuda.memory_allocated(0) / (1024 ** 3):.2f} GB")
print(time.time()-start_time)
start_time = time.time()
print(f"Memory Allocated: {torch.cuda.memory_allocated(0) / (1024 ** 3):.2f} GB")

model.dense_head(inputs_2d)
print(f"Memory Allocated: {torch.cuda.memory_allocated(0) / (1024 ** 3):.2f} GB")

print(time.time()-start_time)

# inputs_bev
# inputs_bev["spatial_features_2d"]

Memory Allocated: 0.24 GB
Memory Allocated: 0.36 GB
Memory Allocated: 0.52 GB
Memory Allocated: 2.91 GB
1.080721139907837
Memory Allocated: 2.91 GB
Memory Allocated: 3.66 GB
0.04155468940734863


In [9]:
# inputs_vfe["spatial_features"]

In [10]:
# inputs_3d.keys()

In [11]:
with torch.no_grad():
    DSVT_2d_Backbone = model.backbone_2d
    # dsvtblocks_list = DSVT_Backbone.stage_0
    # layer_norms_list = DSVT_Backbone.residual_norm_stage_0
    # inputs = model.vfe(inputs)
    # voxel_info = DSVT_Backbone.input_layer(inputs)
    # set_voxel_inds_list = [[voxel_info[f'set_voxel_inds_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
    # set_voxel_masks_list = [[voxel_info[f'set_voxel_mask_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
    # pos_embed_list = [[[voxel_info[f'pos_embed_stage{s}_block{b}_shift{i}'] for i in range(2)] for b in range(4)] for s in range(1)]



In [12]:
input_names = [
    # 'voxel_features',
    # 'use_lead_xyz',
    # 'batch_size',
    # 'pillar_features',
    # 'voxel_coords',
    'spatial_features'
]
output_names = ["output",]
trt_path = './deploy_pillar_sfaw_2d_rev/dsvt_2d.engine'


In [13]:
input_dict = {}
input_dict={}
# input_dict['voxel_features'] = inputs_bev['voxel_features']
# input_dict['use_lead_xyz'] = inputs_bev['use_lead_xyz']
# input_dict['batch_size'] = torch.tensor(inputs_bev['batch_size']).to("cuda:0")
# input_dict['pillar_features'] = inputs_bev['pillar_features']
# input_dict['voxel_coords'] = inputs_bev['voxel_coords']
input_dict['spatial_features'] = inputs_bev['spatial_features']
input_dict

{'spatial_features': tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          ...,
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
   

In [14]:
# output = input_dict['voxel_features']
# outputs = 7*[None]
# outputs[0] = torch.empty(input_dict['voxel_features'].shape).to('cuda:0')
# outputs[1] = torch.empty(input_dict['use_lead_xyz'].shape).to('cuda:0')
# outputs[2] = torch.empty(input_dict['batch_size'].shape).to('cuda:0')
# outputs[3] = torch.empty(input_dict['pillar_features'].shape).to('cuda:0')
# outputs[4] = torch.empty(input_dict['voxel_coords'].shape).to('cuda:0')
# outputs[5] = torch.empty(input_dict['spatial_features'].shape).to('cuda:0')
# outputs[6] = torch.empty(input_dict['spatial_features'].shape).to('cuda:0')
# output = torch.empty(input_dict['spatial_features'].shape).to('cuda:0')
# output = input_dict['spatial_features']

In [15]:
# input_dict['pillar_features'].shape
# engine.get_tensor_profile_shape("output", 0)


In [16]:
# output.size()
# output = torch.empty([10000, 10000]).to('cuda:0')
# set_voxel_inds_list[0][0].int().size()

In [17]:
# tensor([[-1.0312, -0.2234, -0.2581,  ..., -1.2207,  0.0303, -0.2534],
#         [ 0.0000,  0.0686,  0.0000,  ...,  0.0079,  0.0000,  0.0000],
#         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
#         ...,
#         [ 0.5312,  0.4485,  0.0000,  ...,  0.0691,  0.0000,  0.5523],
#         [ 0.0000,  0.4938,  0.0000,  ...,  0.0557,  0.0000,  0.0000],
#         [ 0.0000,  0.4529,  0.0000,  ...,  0.0210,  0.0000,  0.0000]],
#        device='cuda:0', grad_fn=<CloneBackward0>)

In [18]:
def load_trt_engine(path: str) -> trt.ICudaEngine:
    """Deserialize TensorRT engine from disk.

    Args:
        path (str): The disk path to read the engine.

    Returns:
        tensorrt.ICudaEngine: The TensorRT engine loaded from disk.
    """
    # load_tensorrt_plugin()
    with trt.Logger() as logger, trt.Runtime(logger) as runtime:
        with open(path, mode='rb') as f:
            engine_bytes = f.read()
        engine = runtime.deserialize_cuda_engine(engine_bytes)
        print(f"TensorRT engine {path} successfully loaded.")
        return engine

In [19]:
trt.init_libnvinfer_plugins(None, '')
if isinstance(trt_path, str):
    engine = load_trt_engine(trt_path)

if not isinstance(engine, trt.ICudaEngine):
    raise TypeError(f'`engine` should be str or trt.ICudaEngine, \
        but given: {type(engine)}')

context = engine.create_execution_context()

TensorRT engine ./deploy_pillar_sfaw_2d_rev/dsvt_2d.engine successfully loaded.
[04/19/2024-01:32:31] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


In [20]:
# context.set_tensor_address("src", ptr)
engine.get_tensor_profile_shape("output", 0)
output = torch.empty([1, 384, 468, 468]).to('cuda:0')


In [21]:
# inputs.items()
engine.get_tensor_name(0)
# engine.get_tensor_name(1)
# engine.get_tensor_name(2)
# engine.get_tensor_name(3)
# engine.get_tensor_name(4)
# engine.get_tensor_name(5)
# engine.get_tensor_name(6)



'spatial_features'

In [22]:
# output_shape = engine.get_tensor_profile_shape("output", 0)
# output_shape[0]

In [23]:
# src_shape = engine.get_tensor_profile_shape("src", 0)[0]
# output = torch.empty([output_shape[0]]).to('cuda:0')

In [24]:
# engine.get_tensor_profile_values(0,"src")
# input_host_mem = cuda.pagelocked_empty(trt.volume(src_shape) * trt.float32.itemsize)

# bindings = [None] * (len(input_names) + len(output_names))

# bindings

In [25]:
# context.execute_async_v3(
#             bindings,
#             torch.cuda.current_stream().cuda_stream,
#         )


In [26]:
# engine.get_profile_shape(0,"src")
# engine.get_tensor_location("src")
# engine.get_tensor_format("src")
engine.get_tensor_name(1)
# engine.get_tensor_profile_values(0,"src")

'output'

In [27]:
# profile = self.engine.get_profile_shape(profile_id, input_name)
# assert input_tensor.dim() == len(
#     profile[0]), 'Input dim is different from engine profile.'
# for s_min, s_input, s_max in zip(eval(repr(profile[0])), input_tensor.shape,
#                                  eval(repr(profile[2]))):
#     assert s_min <= s_input <= s_max, \
#         f'Input shape of {input_name} should be between ' \
#         + f'{profile[0]} and {profile[2]}' \
#         + f' but get {tuple(input_tensor.shape)}.'

In [28]:
bindings = [None] * (len(input_names) + len(output_names))

# https://github.dev/Haiyang-W/DSVT , dsvt.py -> 384line
profile_id = 0
idx = 0
for input_name, input_tensor in input_dict.items():    
    profile = engine.get_profile_shape(profile_id,input_name)
    if input_tensor.dtype == torch.long:
        input_tensor=input_tensor.int()
        print("convert long to int")
    print(f'input_name: {input_name}')
    print(f'profile : {profile}')
    print(f'type : {input_tensor.dtype}')
    print(f'shape : {input_tensor.size()}')
    print("======")
    context.set_input_shape(input_name,tuple(input_tensor.shape))
    bindings[idx] = input_tensor.contiguous().data_ptr()
    idx+=1
# get_binding_index

input_name: spatial_features
profile : [(1, 192, 468, 468), (1, 192, 468, 468), (1, 192, 468, 468)]
type : torch.float32
shape : torch.Size([1, 192, 468, 468])


/tmp/ipykernel_2355162/4173236747.py:7: DeprecationWarning: Use get_tensor_profile_shape instead.
  profile = engine.get_profile_shape(profile_id,input_name)


In [29]:
bindings

[139733673967616, None]

In [30]:
output.shape
# profile = engine.get_profile_shape(0,"output")

output_shape = profile[0]
# output = 

In [31]:
# context.set_input_shape("output",tuple(output_shape))

bindings[-1] = output.data_ptr()
# for idx, output in enumerate(outputs):
    # print(output.shape)
    # bindings[idx+6] = output.data_ptr()

# for output_name in output_names:
    # print(output_name)
    # idx = self.engine.get_binding_index(output_name)
    # dtype = torch_dtype_from_trt(self.engine.get_binding_dtype(idx))
    # shape = eval(repr(self.context.get_binding_shape(idx)))

    # device = torch_device_from_trt(self.engine.get_location(idx))
    # output = torch.empty(size=shape, dtype=dtype, device=device)
    # outputs[output_name] = output
    # bindings[idx] = output.data_ptr()


In [32]:
bindings

[139733673967616, 139728573693952]

In [33]:
start_time = time.time()
print(f"Memory Allocated: {torch.cuda.memory_allocated(0) / (1024 ** 3):.2f} GB")

context.execute_v2(
            bindings
        )
print(f"Memory Allocated: {torch.cuda.memory_allocated(0) / (1024 ** 3):.2f} GB")

print(time.time()-start_time)

Memory Allocated: 3.22 GB
Memory Allocated: 3.22 GB
0.05564403533935547


In [34]:
output.size()


torch.Size([1, 384, 468, 468])

In [35]:
output

tensor([[[[0.3520, 0.7090, 0.6403,  ..., 0.6391, 0.5351, 0.9621],
          [0.6059, 0.6821, 0.6289,  ..., 0.6430, 0.7770, 1.0879],
          [0.4052, 0.5088, 0.1898,  ..., 0.1131, 0.3867, 0.8935],
          ...,
          [0.4514, 0.6178, 0.3753,  ..., 0.3189, 0.4214, 0.8849],
          [0.5247, 0.6775, 0.6047,  ..., 0.6434, 0.6046, 0.9232],
          [0.5045, 0.6075, 0.5154,  ..., 0.5288, 0.6361, 0.7326]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0208,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

In [36]:
# context.execute_async_v3(bindings[0])

In [37]:
####


In [38]:
# # engine = load_engine(engine_file_path)
# # Load the TensorRT engine from the file
# with open(engine_file_path, 'rb') as f, trt.Runtime(trt.Logger(trt.Logger.WARNING)) as runtime:
#     engine = runtime.deserialize_cuda_engine(f.read())
#     context = engine.create_execution_context()
# # engine


In [39]:
# num_bindings = engine.num_bindings
# binding_info = []
# for i in range(num_bindings):
#     binding_info.append({
#         'index': i,
#         'name': engine.get_binding_name(i),
#         'shape': engine.get_binding_shape(i),
#         'dtype': trt.nptype(engine.get_binding_dtype(i))
#     })

In [40]:
context = engine.create_execution_context()

# 입력 및 출력 버퍼를 할당합니다.
input_shape = engine.get_binding_shape(0)
output_shape = engine.get_binding_shape(1)
input_host_mem = cuda.pagelocked_empty(trt.volume(input_shape) * trt.float32.itemsize)
output_host_mem = cuda.pagelocked_empty(trt.volume(output_shape) * trt.float32.itemsize)
input_device_mem = cuda.mem_alloc(input_host_mem.nbytes)
output_device_mem = cuda.mem_alloc(output_host_mem.nbytes)

# 이미지 전처리
image = preprocess_image(image_path, input_shape)

# 입력을 GPU 메모리로 복사합니다.
cuda.memcpy_htod(input_device_mem, image.flatten())

# 추론 실행
bindings = [int(input_device_mem), int(output_device_mem)]
context.execute_v2(bindings)

# 결과를 GPU에서 CPU로 복사합니다.
cuda.memcpy_dtoh(output_host_mem, output_device_mem)

[04/19/2024-01:32:39] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


AttributeError: 'tensorrt.tensorrt.ICudaEngine' object has no attribute 'get_binding_shape'

In [ ]:
input_dict = {}
input_dict['data_dict']={}
input_dict['data_dict']['voxel_features'] = ort_outs[0]
input_dict['data_dict']['use_lead_xyz'] = ort_outs[1]
input_dict['data_dict']['batch_size'] = ort_outs[2]
input_dict['data_dict']['pillar_features'] = ort_outs[3]
input_dict['data_dict']['voxel_coords'] = ort_outs[4]
input_dict['data_dict']['spatial_features'] = ort_outs[5]


In [ ]:
base_name = "./deploy_pillar_sfaw_2d/dsvt_2d_bb"
ts_path = f"{base_name}.ts"
onnx_path = f"{base_name}.onnx"

ort_session = ort.InferenceSession(onnx_path)

ort_inputs = {
    'voxel_features' : input_dict['data_dict']['voxel_features'],
    'use_lead_xyz' : input_dict['data_dict']['use_lead_xyz'],
    'batch_size' :  input_dict['data_dict']['batch_size'],
    'pillar_features' : input_dict['data_dict']['pillar_features'],
    'voxel_coords' : input_dict['data_dict']['voxel_coords'],
    'spatial_features' : input_dict['data_dict']['spatial_features']
}
ort_outs = ort_session.run(None, ort_inputs)
ort_outs


In [ ]:
#### Head

In [ ]:
# data_dict = {}
input_dict = {}
input_dict['data_dict']={}
input_dict['data_dict']['voxel_features'] = ort_outs[0]
input_dict['data_dict']['use_lead_xyz'] = ort_outs[1]
input_dict['data_dict']['batch_size'] = ort_outs[2]
input_dict['data_dict']['pillar_features'] = ort_outs[3]
input_dict['data_dict']['voxel_coords'] = ort_outs[4]
input_dict['data_dict']['spatial_features'] = ort_outs[5]
input_dict['data_dict']['spatial_features_2d'] = ort_outs[6]



In [ ]:
# # test onnx
base_name = "./deploy_pillar_sfaw_head/dsvt_head"
ts_path = f"{base_name}.ts"
onnx_path = f"{base_name}.onnx"


ort_session = ort.InferenceSession(onnx_path)

ort_inputs = {
    'voxel_features' : input_dict['data_dict']['voxel_features'],
    'use_lead_xyz' : input_dict['data_dict']['use_lead_xyz'],
    'batch_size' :  input_dict['data_dict']['batch_size'],
    'pillar_features' : input_dict['data_dict']['pillar_features'],
    'voxel_coords' : input_dict['data_dict']['voxel_coords'],
    'spatial_features' : input_dict['data_dict']['spatial_features'],
    'spatial_features_2d' : input_dict['data_dict']['spatial_features_2d']
}

ort_outs = ort_session.run(None, ort_inputs)
ort_outs